***Importing the libraries***

In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.tsv', sep = '\t')

In [3]:
train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


***Extract only those movie reviews which have extremely positive/negative sentiments***

In [4]:
train[(train['Sentiment'] == 0) | (train['Sentiment'] == 4)]

,PhraseId,SentenceId,Phrase,Sentiment
63,64,2,"This quiet , introspective and entertaining in...",4
66,67,2,"quiet , introspective and entertaining indepen...",4
74,75,2,entertaining,4
77,78,2,is worth seeking,4
101,102,3,would have a hard time sitting through this one,0
...,...,...,...,...
155970,155971,8539,"of unfocused , excruciatingly tedious cinema",0
155971,155972,8539,"unfocused , excruciatingly tedious cinema",0
155973,155974,8539,", excruciatingly tedious",0
156007,156008,8541,really do a great job of anchoring the charact...,4


In [5]:
train = train[(train['Sentiment'] == 0) | (train['Sentiment'] == 4)]
train['Rating'] = train.apply(lambda x: 0 if x['Sentiment'] == 0 else 1, axis = 1)

In [6]:
from sklearn.model_selection import train_test_split

X = train['Phrase']
y = train['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_final = vectorizer.fit_transform(X_train)

print(X_train_final.shape)

(11394, 9679)


In [8]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
clf.fit(X_train_final, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

***Alternatively, we can use pipelines to summarize this work***

In [9]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                      ('clf', LinearSVC()),
                    ])
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [10]:
predictions = text_clf.predict(X_test)

In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.9447174447174447
[[1994  148]
 [ 122 2620]]
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      2142
           1       0.95      0.96      0.95      2742

    accuracy                           0.94      4884
   macro avg       0.94      0.94      0.94      4884
weighted avg       0.94      0.94      0.94      4884



In [12]:
import pickle
    
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
    
with open('sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)